# Problem Set 2: Two-player Iterated Prisoner's Dilemma
Minho Kang (239742)

## Introduction
In this problem set, I will simulate a two-player Iterated Prisoner's Dilemma (IPD) game using several strategies. The objective of this assignment is to explore how different strategies interact in the context of the prisoner's dilemma, focusing on payoffs, cooperation, and defection patterns. We will test the following strategies:

1. Tit-for-Tat
2. Grim Trigger
3. Always Cooperate
4. Always Defect
5. Probabilistic Strategy
6. Intermediate Punishment Strategy

### The Payoff Matrix
The payoff matrix used for the Iterated Prisoner's Dilemma is as follows:

|                               | **Cooperate** | **Don't cooperate** |
|-------------------------------|---------------|---------------------|
| **Cooperate**                  | 6, 6          | 2, 10               |
| **Don't cooperate**            | 10, 2         | 4, 4                |

## Objective
The primary objective of this problem set is to implement and simulate a series of strategies in an iterated version of the Prisoner's Dilemma game and analyze their payoffs and behavior when played against each other.

## Methodology
### Strategies Implemented
1. **Tit-for-Tat**: This strategy cooperates on the first move and then mimics the opponent's previous move.
2. **Grim Trigger**: This strategy cooperates until the opponent defects, after which it defects permanently.
3. **Always Cooperate**: This strategy always cooperates regardless of the opponent's moves.
4. **Always Defect**: This strategy always defects regardless of the opponent's moves.
5. **Probabilistic Strategy**: This strategy cooperates with a given probability `p` and defects with the complementary probability.
6. **Intermediate Punishment**: This strategy cooperates until the opponent defects. If the opponent defects, it will not cooperate for the next `k` rounds but will return to cooperation afterward.

### Simulation Setup
The game was simulated over a series of 10 rounds, where each strategy played against every other strategy, except against itself. For each game, the cumulative payoffs were calculated, and the moves of the players were recorded for visualization.

The number of rounds was chosen to be 10 to balance between observing long-term effects and avoiding excessive computation time. Additionally, the `k` parameter in the Intermediate Punishment strategy was set to 3.


In [1]:
import os
from ps2.strategies import *
from ps2.save_plot import save_plot
from ps2.simulate_game import simulate_game

In [2]:
payoff_matrix = {
    ("C", "C"): (6, 6),
    ("C", "D"): (2, 10),
    ("D", "C"): (10, 2),
    ("D", "D"): (4, 4),
}

In [3]:
# set a
save_dir = "/Users/minhokang/hertie/agt-fall25/ps2/plots"
os.makedirs(save_dir, exist_ok=True)

In [6]:
# create a list of tuple for strategy names and their functions
strategies = [
    ("Tit-for-Tat", tit_for_tat),
    ("Grim Trigger", grim_trigger),
    ("Always Cooperate", always_cooperate),
    ("Always Defect", always_defect),
    ("Probabilistic (p=0.5)", lambda h: probabilistic_strategy(h, p=0.5)),
    ("Intermediate Punishment", intermediate_punishment),
]

# simulate and save results for all strategy combinations
payoffs = []  # To store payoff results
for i in range(len(strategies)):
    for j in range(i + 1, len(strategies)):  # i to j to avoid duplicate pairs
        strategy1_name, strategy1 = strategies[i]
        strategy2_name, strategy2 = strategies[j]

        # Simulate the game
        player1_moves, player2_moves, player1_payoff, player2_payoff = simulate_game(
            player1_strategy=strategy1, player2_strategy=strategy2, rounds=10, payoff_matrix=payoff_matrix
        )

        # Save the plot and calculate payoffs
        save_plot(
            player1_strategy_name=strategy1_name,
            player2_strategy_name=strategy2_name,
            player1_moves=player1_moves,
            player2_moves=player2_moves,
            player1_payoff=player1_payoff,
            player2_payoff=player2_payoff,
            save_dir=save_dir,
        )

        # Store the payoffs for later analysis
        payoffs.append((strategy1_name, strategy2_name, player1_payoff, player2_payoff))


In [7]:
print("Payoff Summary:")
for payoff in payoffs:
    print(f"{payoff[0]} vs {payoff[1]}: {payoff[2]} - {payoff[3]}")

Payoff Summary:
Tit-for-Tat vs Grim Trigger: 60 - 60
Tit-for-Tat vs Always Cooperate: 60 - 60
Tit-for-Tat vs Always Defect: 38 - 46
Tit-for-Tat vs Probabilistic (p=0.5): 50 - 50
Tit-for-Tat vs Intermediate Punishment: 60 - 60
Grim Trigger vs Always Cooperate: 60 - 60
Grim Trigger vs Always Defect: 38 - 46
Grim Trigger vs Probabilistic (p=0.5): 64 - 40
Grim Trigger vs Intermediate Punishment: 60 - 60
Always Cooperate vs Always Defect: 20 - 100
Always Cooperate vs Probabilistic (p=0.5): 36 - 84
Always Cooperate vs Intermediate Punishment: 60 - 60
Always Defect vs Probabilistic (p=0.5): 64 - 32
Always Defect vs Intermediate Punishment: 46 - 38
Probabilistic (p=0.5) vs Intermediate Punishment: 48 - 64
